In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
file_path = './Model.xlsx'
sheet_name = 'Model'
data = pd.read_excel(file_path, sheet_name=sheet_name)

In [3]:
print(data.columns)

Index(['TOTEX', 'TOINC', 'URB', 'FSIZE', 'emp_status', 'RPROV'], dtype='object')


In [4]:
X_Categorical = data[["URB", "emp_status"]]
X_Num = data[["TOINC", "FSIZE"]]
y = data["TOTEX"]

In [5]:
X_Categorical_encoded = pd.get_dummies(X_Categorical, drop_first=True)

In [6]:
X = pd.concat([X_Num, X_Categorical_encoded], axis=1)

In [7]:
corr_matrix = data.corr()

# Display the correlation matrix
print("Correlation Matrix:")
print(corr_matrix)

Correlation Matrix:
               TOTEX     TOINC       URB     FSIZE  emp_status     RPROV
TOTEX       1.000000  0.761097 -0.253112  0.213179    0.128642  0.072926
TOINC       0.761097  1.000000 -0.174783  0.137619    0.101858  0.034538
URB        -0.253112 -0.174783  1.000000  0.007064   -0.104917 -0.160591
FSIZE       0.213179  0.137619  0.007064  1.000000    0.196903 -0.001118
emp_status  0.128642  0.101858 -0.104917  0.196903    1.000000 -0.004863
RPROV       0.072926  0.034538 -0.160591 -0.001118   -0.004863  1.000000


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [10]:
predictions = model.predict(X_test)

In [11]:
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})
print("Model Coefficients:")
print(coefficients)

r2_score = model.score(X_test, y_test)
print(f"R^2 Score on Test Set: {r2_score}")


Model Coefficients:
      Feature   Coefficient
0       TOINC      0.331326
1       FSIZE  11394.709144
2         URB -49533.816362
3  emp_status  11159.293812
4       RPROV      2.286073
R^2 Score on Test Set: 0.6852167333755599


In [13]:
y_pred = model.predict(X_test)
comparison_df = pd.DataFrame({
    "Actual": y_test.values,
    "Predicted (Scikit-learn)": y_pred
})

# Optionally, calculate residuals
comparison_df["Residuals (Scikit-learn)"] = comparison_df["Actual"] - comparison_df["Predicted (Scikit-learn)"]


print(comparison_df)

          Actual  Predicted (Scikit-learn)  Residuals (Scikit-learn)
0      205230.00             172213.804100              33016.195900
1      110389.00             246387.630452            -135998.630452
2      156883.00             187345.185818             -30462.185818
3      128421.00             138915.653934             -10494.653934
4       91099.00             122524.709691             -31425.709691
...          ...                       ...                       ...
32649  174550.00             180148.007321              -5598.007321
32650   80208.00             166808.600782             -86600.600782
32651  112159.00             128542.066190             -16383.066190
32652   82453.75             135814.522253             -53360.772253
32653  194223.00             218705.979666             -24482.979666

[32654 rows x 3 columns]


In [118]:
comparison_df.to_csv('predictions.csv', index=False)